In [61]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, f1_score
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler

from imblearn.over_sampling import SMOTE
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.utils import to_categorical
import matplotlib.pyplot as plt
import seaborn as sns




# 1. Traitement des données 

Importation du jeu de données et division en données classe et caractéristique

In [62]:
dt = pd.read_csv("/Users/mallorylecorre/Desktop/Dm_Ia/cars_data-class.csv")
print(dt.head())

X = dt.drop('Impact_Environnement', axis=1)
y = dt['Impact_Environnement']

   Taille_Auto   Poids_Auto  Acceleration  Annee_Fabrication    Kilometrage  \
0     3.790317  8869.946374      2.038688               2006  135542.067972   
1     3.375892  1126.174079      2.067411               2010   33710.551662   
2     4.844971  6100.019066      4.839676               2016  112977.279109   
3     2.316172  5771.509867      1.202574               2014   50139.727635   
4     2.618825  5676.666211      5.470806               2009  191188.750210   

   Nombre_Portes  Vitesse_Max  Prix_Assurance  Categorie_Voiture    Decote  \
0              5   195.289066      184.125577                  1  0.104588   
1              5   249.979656      858.935088                  3  0.346064   
2              5   235.763320      302.138838                  4  0.394582   
3              4   216.802020     1298.671966                  5  0.422048   
4              4   234.004505      702.626517                  2  0.279959   

   Impact_Environnement  
0                     1  
1   

In [51]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


On standardise les données pour mettre toute les variables à la même échelle.

In [75]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

Distribution des données :

In [76]:
dt['Impact_Environnement'].value_counts()

1    537
3    385
2     42
0     36
Name: Impact_Environnement, dtype: int64

On peut voir que le jeu de données est déséquilibrée avec une sur-représentation de la classe 1.

On équilibre les données en sur-échantillonant avec SMOTE

In [77]:
smote = SMOTE(random_state=42)
X_train, y_train = smote.fit_resample(X_train, y_train)


On vérifie que le rééquillibrage a bien fonctionné

In [78]:
print("Nouvelle distribution :")
print(pd.Series(y_train).value_counts())

Nouvelle distribution :
1    428
3    428
0    428
2    428
Name: Impact_Environnement, dtype: int64


Les données sont équilibrées

# Création du modèle 

In [79]:
model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(32, activation='sigmoid'))
model.add(Dense(4, activation='softmax'))

compilation du modèle en tuilisant une fonction de perte qui correspond à une classification multi-class

In [80]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


Ajout d'une cross validatoin

Entrainement du modèle 

In [81]:
model.fit(X_train, y_train, epochs=50, batch_size=40, validation_split=0.2)


Epoch 1/50
35/35 [==============================] - 0s 3ms/step - loss: 1.3909 - accuracy: 0.2834 - val_loss: 1.5060 - val_accuracy: 0.0496
Epoch 2/50
35/35 [==============================] - 0s 960us/step - loss: 1.2059 - accuracy: 0.5252 - val_loss: 1.5221 - val_accuracy: 0.2216
Epoch 3/50
35/35 [==============================] - 0s 913us/step - loss: 1.0913 - accuracy: 0.6435 - val_loss: 1.4079 - val_accuracy: 0.2857
Epoch 4/50
35/35 [==============================] - 0s 972us/step - loss: 0.9891 - accuracy: 0.6954 - val_loss: 1.3190 - val_accuracy: 0.2886
Epoch 5/50
35/35 [==============================] - 0s 943us/step - loss: 0.9068 - accuracy: 0.7173 - val_loss: 1.2609 - val_accuracy: 0.2886
Epoch 6/50
35/35 [==============================] - 0s 977us/step - loss: 0.8433 - accuracy: 0.7210 - val_loss: 1.1541 - val_accuracy: 0.2974
Epoch 7/50
35/35 [==============================] - 0s 1ms/step - loss: 0.7953 - accuracy: 0.7297 - val_loss: 1.1166 - val_accuracy: 0.3090
Epoch 8/50

# Evalutation du modèle

In [86]:
y_pred_probabilities = model.predict(X_test)
y_pred = np.argmax(y_pred_probabilities, axis=1)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

7/7 [==============================] - 0s 553us/step
Accuracy: 0.795


affichaqge des différents score et de la matrice de confusion